<a href="https://colab.research.google.com/github/Thekowo/InfoLing-SchenkKlitschkov/blob/main/Crypto_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Make sure you have a GPU running
!nvidia-smi

Tue Sep 21 15:52:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git


  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-ci4afflz
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-ci4afflz


In [3]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [4]:
# Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

09/21/2021 15:52:20 - INFO - haystack.utils -   Starting Elasticsearch ...
09/21/2021 15:52:20 - WARNING - haystack.utils -   Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


In [5]:
# In Colab / No Docker environments: Start Elasticsearch from source
#! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
#! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
#! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

FileNotFoundError: ignored

In [ ]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
document_store.delete_all_documents



In [ ]:
# txt. files in dem angegeben pfad ablegen!
path = 'data/crypto'
wrong_path= 'data/crypto/txt_files/~BROMIUM'
from zipfile import ZipFile
with ZipFile('txt_files.zip', 'r') as zipObj:
  zipObj.extractall(path)
import os
import sys
import shutil

# Get directory name
mydir= wrong_path

try:
    shutil.rmtree(mydir)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))
docs = path

# Convert files to dicts, with the already pre defined clean_wiki_text cleaning funciton
dicts = convert_files_to_dicts(dir_path=docs, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.

# Write the dicts containing documents to our DB.
document_store.write_documents(dicts)

In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
# Load a model that is suitable for the task - we chose roberta-base-squad2)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [ ]:
# Alternative: Shit
reader2 = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

In [ ]:
# ExtractiveQAPipeline lets you specify a retriever and a reader
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
answers= []
for i in range(x):
  answers[i]=pipe.run(query=questions[i], params= {"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
    )
#prediction = pipe.run(query="why to use bitcoin", params= {"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
#    )

In [ ]:
# prediction = pipe.run(query="Who created the Dothraki vocabulary?", top_k_reader=5)
# prediction = pipe.run(query="Who is the sister of Sansa?", top_k_reader=5)

In [ ]:
print_answers(prediction, details="minimal")